<a href="https://colab.research.google.com/github/DayenaJeong/FS_neruon/blob/main/FS_conversion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def relu(x):
    return np.maximum(0, x)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def swish(x):
    return x * 1 / (1 + np.exp(-x))


def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))


def softplus(x, beta=1):
    return (1 / beta) * np.log(1 + np.exp(beta * x))


def mish(x, beta=1):
    return x * np.tanh(softplus(x, beta))

In [4]:
# Implementation of spike function for PyTorch custom gradient
class SpikeFunction(autograd.Function):
    @staticmethod
    def forward(ctx, v_scaled):
        z_ = torch.where(v_scaled > 0, torch.ones_like(v_scaled), torch.zeros_like(v_scaled))
        ctx.save_for_backward(v_scaled)
        return z_

    @staticmethod
    def backward(ctx, grad_output):
        v_scaled, = ctx.saved_tensors
        dz_dv_scaled = torch.maximum(1 - torch.abs(v_scaled), torch.tensor(0.0, device=v_scaled.device))
        dE_dv_scaled = grad_output * dz_dv_scaled
        return dE_dv_scaled

# Call spike function for PyTorch
def spike_function(v_scaled):
    return SpikeFunction.apply(v_scaled)

# FS class definition
class FS(nn.Module):
    def __init__(self, input_dim, k_neurons, num_params):
        super(FS, self).__init__()
        self.k_neurons = k_neurons

        # Define FS parameters (now as learnable parameters)
        self.h = nn.Parameter(torch.abs(torch.randn(num_params)))
        self.d = nn.Parameter(torch.abs(torch.randn(num_params)))
        self.T = nn.Parameter(torch.randn(num_params))

        # Add trainable parameters
        self.linear = nn.Linear(input_dim, k_neurons)

        # Additional linear layer for final output
        self.final_linear = nn.Linear(k_neurons, 1)

    def forward(self, x):
        # Apply linear transformation
        v = self.linear(x)

        # Initialize temporary output for FS spike neural network
        temp_out = torch.zeros_like(v)

        # Implement FS spike neural network
        for t in range(len(self.T)):
            v_scaled = (v - self.T[t]) / (torch.abs(v) + 1)
            z = spike_function(v_scaled)
            temp_out += z * self.d[t]
            v = v - z * self.h[t]

        # Apply additional linear layer for final output
        out = self.final_linear(temp_out)

        return out

Sigmoid

In [ ]:
# Instantiate model and initial setup
input_dim = 1
k_neurons = 16  # Select arbitrary number of neurons
num_params = 16  # Select arbitrary number of parameters
model = FS(input_dim, k_neurons, num_params)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training data
x_train = torch.linspace(-5, 5, steps=100).unsqueeze(1)  # Input
y_train = sigmoid(x_train)  # Target value

torch.autograd.set_detect_anomaly(True)

# Training loop
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()  # Reset gradients
    y_pred = model(x_train)  # Forward pass
    loss = criterion(y_pred, y_train)  # Remove `.squeeze()` call
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Print loss every 100 epochs
    if epoch % 100 == 0:
       print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

# Print final loss
print(f'Final Loss: {loss.item()}')

# Calculate model predictions
with torch.no_grad():  # No gradient computation needed
    y_pred = model(x_train).squeeze()

# True function values
y_true = sigmoid(x_train).squeeze()

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(x_train.numpy(), y_true.numpy(), label='True Sigmoid Function', color='r')
plt.plot(x_train.numpy(), y_pred.numpy(), label='Model Prediction', linestyle='--', color='b')
plt.title('Comparison between True Sigmoid Function and Model Prediction')
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.legend()

plt.show()

GeLU

In [ ]:
# Instantiate model and initial setup
input_dim = 1
k_neurons = 16  # Select arbitrary number of neurons
num_params = 16  # Select arbitrary number of parameters
model = FS(input_dim, k_neurons, num_params)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training data
x_train = torch.linspace(-10, 10, steps=100).unsqueeze(1)  # Input
y_train = gelu(x_train)  # Target value

torch.autograd.set_detect_anomaly(True)

# Training loop
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()  # Reset gradients
    y_pred = model(x_train)  # Forward pass
    loss = criterion(y_pred, y_train)  # Remove `.squeeze()` call
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Print loss every 100 epochs
    if epoch % 100 == 0:
       print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

# Print final loss
print(f'Final Loss: {loss.item()}')

# Calculate model predictions
with torch.no_grad():  # No gradient computation needed
    y_pred = model(x_train).squeeze()

# True function values
y_true = gelu(x_train).squeeze()

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x_train.numpy(), y_true.numpy(), label='True GeLU Function', color='r')
plt.plot(x_train.numpy(), y_pred.numpy(), label='Model Prediction', linestyle='--', color='b')
plt.title('Comparison between True GeLU Function and Model Prediction')
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.legend()

plt.show()

In [ ]:
# Instantiate model and initial setup
input_dim = 1
k_neurons = 4  # Select arbitrary number of neurons
num_params = 4  # Select arbitrary number of parameters
model = FS(input_dim, k_neurons, num_params)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training data
x_train = torch.linspace(-10, 10, steps=100).unsqueeze(1)  # Input
y_train = softplus(x_train)  # Target value

torch.autograd.set_detect_anomaly(True)

loss_values = []

# Training loop
epochs = 100000
for epoch in range(epochs):
    optimizer.zero_grad()  # Reset gradients
    y_pred = model(x_train)  # Forward pass
    loss = criterion(y_pred, y_train)  # Remove `.squeeze()` call
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
       print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

    if epoch % 200 == 0:
       loss_values.append(loss.item())

# Print final loss
print("Loss values:", loss_values)
print(f'Final Loss: {loss.item()}')
print(f'Final h: {model.h.data}')
print(f'Final d: {model.d.data}')
print(f'Final T: {model.T.data}')

# Calculate model predictions
with torch.no_grad():  # No gradient computation needed
    y_pred = model(x_train).squeeze()

# True function values
y_true = softplus(x_train).squeeze()

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(x_train.numpy(), y_true.numpy(), label='True Softplus Function', color='r')
plt.plot(x_train.numpy(), y_pred.numpy(), label='Model Prediction', linestyle='--', color='b')
plt.title('Comparison between True Softplus Function and Model Prediction(K=4)')
plt.xlabel('Input x')
plt.ylabel('Output y')
plt.legend()

plt.show()

Epoch 1/100000, Loss: 22.307052612304688
Epoch 1001/100000, Loss: 0.10722210258245468
Epoch 2001/100000, Loss: 0.06375560164451599
Epoch 3001/100000, Loss: 0.048473551869392395
Epoch 4001/100000, Loss: 0.04832608997821808
Epoch 5001/100000, Loss: 0.04657797887921333
Epoch 6001/100000, Loss: 0.050552286207675934
